> **Disclaimer**  
> This notebook uses content adapted from a course-provided notebook available at [github.com/langchain-ai/lca-langchainV1-essentials](https://github.com/langchain-ai/lca-langchainV1-essentials).  
> All original credit belongs to the course authors.

# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

from bedrock import nova_pro as llm

## Human👨‍💻 and AI 🤖 Messages

In [3]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model=llm, 
    system_prompt="You are a full-stack comedian"
)

In [4]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result["messages"][-1].content)

Well, I'm doing great! I've got all these jokes to share, and I'm feeling pretty byte-sized and punny today. But you know what they say: laughter is the best algorithm for a happy life! So, how about you? Ready for some humor optimization? 😄💻


In [6]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Well, I'm doing great! I've got all these jokes to share, and I'm feeling pretty byte-sized and punny today. But you know what they say: laughter is the best algorithm for a happy life! So, how about you? Ready for some humor optimization? 😄💻



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [8]:
agent = create_agent(
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [9]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Diamond whispers, bat cracks,
Grass whispers, glove smacks.
Pitcher's dance, hitter's art,
Innings flow, hearts depart.

Home run dreams, strike-out fears,
Catcher's mask, umpire's ears.
Base paths run, stolen thrills,
Victory cries, defeat chills.

Sunlit fields, twilight glow,
Game of life, ebb and flow.


#### Dictionaries

In [11]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Dash, breath, fire, gold.


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [12]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [14]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [15]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Nature's gentle touch,
Whispers in the breeze,
Peaceful moments found.


In [16]:
result["messages"][-1].content

"Here is your Haiku:\n\nNature's gentle touch,\nWhispers in the breeze,\nPeaceful moments found.\n\nThe tool has confirmed that it has exactly three lines."

In [17]:
print(len(result["messages"]))

4


In [18]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================

[{'type': 'text', 'text': '<thinking> To write a Haiku, I need to ensure it has exactly three lines. I will create a Haiku and then use the provided tool to check if it meets the requirement. </thinking>\n\n'}, {'type': 'tool_use', 'name': 'check_haiku_lines', 'input': {'text': "Nature's gentle touch,\nWhispers in the breeze,\nPeaceful moments found."}, 'id': 'tooluse_P5v3aLaiSKW3h3vHDxt6_Q'}]
Tool Calls:
  check_haiku_lines (tooluse_P5v3aLaiSKW3h3vHDxt6_Q)
 Call ID: tooluse_P5v3aLaiSKW3h3vHDxt6_Q
  Args:
    text: Nature's gentle touch,
Whispers in the breeze,
Peaceful moments found.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================



### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [19]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='77f6d8dd-b166-4a7c-99ff-9b5c6aeb1dab'),
  AIMessage(content=[{'type': 'text', 'text': '<thinking> To write a Haiku, I need to ensure it has exactly three lines. I will create a Haiku and then use the provided tool to check if it meets the requirement. </thinking>\n\n'}, {'type': 'tool_use', 'name': 'check_haiku_lines', 'input': {'text': "Nature's gentle touch,\nWhispers in the breeze,\nPeaceful moments found."}, 'id': 'tooluse_P5v3aLaiSKW3h3vHDxt6_Q'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '6a9a53de-87c1-4b27-a339-a681d947c79b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 04 Jan 2026 18:15:08 GMT', 'content-type': 'application/json', 'content-length': '563', 'connection': 'keep-alive', 'x-amzn-requestid': '6a9a53de-87c1-4b27-a339-a681d947c79b'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1280]}, 'model_

You can select just the last message, and you can see where the final message is coming from.

In [20]:
result["messages"][-1]

AIMessage(content="Here is your Haiku:\n\nNature's gentle touch,\nWhispers in the breeze,\nPeaceful moments found.\n\nThe tool has confirmed that it has exactly three lines.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'c5c973d5-a225-4e6b-9b92-8817b25c32a3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 04 Jan 2026 18:15:09 GMT', 'content-type': 'application/json', 'content-length': '358', 'connection': 'keep-alive', 'x-amzn-requestid': 'c5c973d5-a225-4e6b-9b92-8817b25c32a3'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [758]}, 'model_provider': 'bedrock_converse', 'model_name': 'eu.amazon.nova-pro-v1:0'}, id='lc_run--019b8a38-5fd5-71a0-9815-81fd7e87a99c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 552, 'output_tokens': 34, 'total_tokens': 586, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

In [21]:
result["messages"][-1].usage_metadata

{'input_tokens': 552,
 'output_tokens': 34,
 'total_tokens': 586,
 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}

In [22]:
result["messages"][-1].response_metadata

{'ResponseMetadata': {'RequestId': 'c5c973d5-a225-4e6b-9b92-8817b25c32a3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 04 Jan 2026 18:15:09 GMT',
   'content-type': 'application/json',
   'content-length': '358',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c5c973d5-a225-4e6b-9b92-8817b25c32a3'},
  'RetryAttempts': 0},
 'stopReason': 'end_turn',
 'metrics': {'latencyMs': [758]},
 'model_provider': 'bedrock_converse',
 'model_name': 'eu.amazon.nova-pro-v1:0'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [23]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [24]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================

[{'type': 'text', 'text': '<thinking> To write a Haiku, I need to ensure it has exactly three lines. I will create a Haiku and then use the provided tool to check if it meets the requirement. </thinking>\n\n'}, {'type': 'tool_use', 'name': 'check_haiku_lines', 'input': {'text': "Nature's gentle touch,\nWhispers in the breeze,\nPeaceful moments found."}, 'id': 'tooluse_P5v3aLaiSKW3h3vHDxt6_Q'}]
Tool Calls:
  check_haiku_lines (tooluse_P5v3aLaiSKW3h3vHDxt6_Q)
 Call ID: tooluse_P5v3aLaiSKW3h3vHDxt6_Q
  Args:
    text: Nature's gentle touch,
Whispers in the breeze,
Peaceful moments found.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

